# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
output_data_file = "../output_data/weather_df.csv"


types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,rikitea,91,PF,1586662996,77,-23.12,-134.97,78.55,7.20
1,lagoa,0,PT,1586662716,78,39.05,-27.98,60.85,10.20
2,carnarvon,7,AU,1586662732,69,-24.87,113.63,82.40,20.80
3,talnakh,40,RU,1586662997,79,69.49,88.40,23.00,35.79
4,tasiilaq,99,GL,1586662997,58,65.61,-37.64,26.60,17.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
gmaps.configure(api_key=g_key)

In [27]:
locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df["Humidity"].astype(float)
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
new_types_df = pd.DataFrame(weather_data_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
86,najran,77.00,6.93,0
99,tessalit,77.38,4.32,0
275,baglung,79.12,1.21,0
282,khandbari,78.73,3.51,0
288,guerrero negro,72.57,2.51,0
313,flinders,71.01,3.36,0
428,baracoa,79.48,9.51,0
522,dharchula,73.85,2.33,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = pd.DataFrame(weather_data_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,rikitea,PF,-23.12,-134.97,
1,lagoa,PT,39.05,-27.98,
2,carnarvon,AU,-24.87,113.63,
3,talnakh,RU,69.49,88.40,
4,tasiilaq,GL,65.61,-37.64,
...,...,...,...,...,...
529,almeirim,BR,-1.52,-52.58,
530,mucurapo,TT,10.67,-61.53,
531,bull savanna,LR,6.20,-10.28,
532,san ramon,ML,13.30,-4.90,


In [30]:


target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"


params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}


base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


response = requests.get(base_url, params=params)

In [31]:
hotel_name = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]

    
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

   
    name_address = requests.get(base_url, params=params)
    
    
    name_address = name_address.json()
    
    try:
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [32]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')

In [33]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig



Figure(layout=FigureLayout(height='420px'))